In [6]:
import tensorflow as tf
from keras import layers,optimizers,datasets,Sequential

In [7]:
layer = tf.keras.layers.Conv2D(4,kernel_size = 5 , strides =1 , padding = 'valid')
#4:卷积核个数  padding也以为same

In [8]:
pool = tf.keras.layers.MaxPool2D(2,strides=2)
#2:kernel_size:2 , 

In [9]:
layer = tf.keras.layers.UpSampling2D(size=3)

In [11]:
net = layers.BatchNormalization()
x = tf.random.normal([2,3])
out = net(x)
'''
<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.3583169, -1.3122452, -1.2463319],
        [-1.2777916, -1.7581975, -0.5057919]], dtype=float32)>
'''

In [14]:
net.trainable_variables
'''
[<tf.Variable 'batch_normalization/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
<tf.Variable 'batch_normalization/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]
'''

net.variables
'''
[<tf.Variable 'batch_normalization/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_variance:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>]
'''

[<tf.Variable 'batch_normalization/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization/moving_variance:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>]

In [17]:
x = tf.random.normal([2,4,4,3],mean=-1,stddev=0.5)
net = layers.BatchNormalization(axis=3)
out = net(x,training=True)
net.variables
'''
[<tf.Variable 'batch_normalization_2/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/moving_mean:0' shape=(3,) dtype=float32, numpy=array([-0.00855916, -0.01100943, -0.00956823], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.99258375, 0.991314  , 0.9923488 ], dtype=float32)>]
'''
for i in range(100): out = net(x,training=True)

net.variables
'''
[<tf.Variable 'batch_normalization_1/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_mean:0' shape=(3,) dtype=float32, numpy=array([-0.6259307, -0.58605  , -0.6089093], dtype=float32)>,
 <tf.Variable 'batch_normalization_1/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.48389903, 0.50812393, 0.61608636], dtype=float32)>]
'''






[<tf.Variable 'batch_normalization_2/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/moving_mean:0' shape=(3,) dtype=float32, numpy=array([-0.00855916, -0.01100943, -0.00956823], dtype=float32)>,
 <tf.Variable 'batch_normalization_2/moving_variance:0' shape=(3,) dtype=float32, numpy=array([0.99258375, 0.991314  , 0.9923488 ], dtype=float32)>]

In [ ]:
# for i in range(10):
#     with tf.GradientTape() as tape:
#         out = net(x,training=True)
#         loss = tf.reduce_mean(tf.pow(out,2)) -1 
#     grads = tape.gradient(loss,net.trainable_variables)
#     optimizer.apply_gradients(zip(grads,net.trainable_variables))

In [ ]:
class BasicBlock(layers.Layer):
    def __init__(self,filter_num,stride = 1):
        self.conv1 = layers.Conv2D(filter_num,(3,3),strides=stride,padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')
        self.conv2 = layers.Conv2D(filter_num,(3,3),strides = 1,padding='same')
        self.bn2 = layers.BatchNormalization()
        
        #此步就是短接层，目的就是让两者的特征图可以相加！！！！
        if stride != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num,(1,1),strides=stride))
            self.downsample.add(layers.BatchNormalization())
        else:
            self.downsample = lambda x:x
        self.stride = stride
    def call(self,inputs,training=None):
        residual = self.downsample(inputs)
        conv1 = self.conv1(inputs)
        bn1 = self.bn1(conv1)
        relu = self.relu(bn1)
        conv2 = self.conv2(relu)
        bn2 = self.bn2(conv2)
        
        add = layers.add([bn2,residual])
        out = self.relu(add)
        return out